### AUTOENCODERS FOR GENERATOR
##### convolutional autoencoder, variational autoencoder, u-net, residual u-net, recurrent u-net, r2u-net

In [ ]:
import sys
sys.path.append('../')
import src

In [ ]:
from os import *
import numpy as np
import sys
import re
import math
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from  matplotlib.animation import FuncAnimation
from matplotlib import colors
from netCDF4 import Dataset
from IPython.display import clear_output
#data folder
sys.path.insert(0, '/Users/jlee/Desktop/JONG/TUM/18W/\
Advanced_Deep_Learning_for_Computer_Vision/project/data')

In [ ]:
#forces CPU usage
environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
environ["CUDA_VISIBLE_DEVICES"] = "-1" #"" or "-1" for CPU, "0" for GPU
import tensorflow as tf
from tensorflow import keras
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
inputs = src.get_data(sys.path[0]+"/RW-201008", total_length=101)

In [ ]:
# images = src.generate_datasets(inputs, n=1000, size=64, length=2, normalize=True, split=(6,2,2))
images = np.load(sys.path[0]+"/dataset.npy").item()
# images

In [ ]:
#low_res_train = images["low_res_train"]
#low_res_xval = images["low_res_xval"]
#low_res_test = images["low_res_test"]
#overfit = np.reshape(images['images'],np.shape(images['images'])+(1,))
train = np.reshape(images["train"],np.shape(images["train"])+(1,))
xval = np.reshape(images["xval"],np.shape(images["xval"])+(1,))
test = np.reshape(images["test"],np.shape(images["test"])+(1,))
print(f"Training data: {train.shape}\nValidation data: {xval.shape}\nTest data: {test.shape}")

In [ ]:
%matplotlib inline
src.visualise_data(train[:,:,:,:,0])

In [ ]:
train[:,0,:,:,:].shape, xval[:,0,:,:,:].shape

In [ ]:
overfit_train = np.reshape(train[5,0,:,:,:],((1,)+train.shape[2:]))
overfit_truth = np.reshape(train[5,1,:,:,:],((1,)+train.shape[2:]))
overfit_train.shape

In [ ]:
images_5min = np.load(sys.path[0]+"/5_minute.npy").item()
train_5min = np.reshape(images_5min["train"],np.shape(images_5min["train"])+(1,))
val_5min = np.reshape(images_5min["xval"],np.shape(images_5min["xval"])+(1,))
test_5min = np.reshape(images_5min["test"],np.shape(images_5min["test"])+(1,))
print(f"Training data: {train.shape}\nValidation data: {xval.shape}\nTest data: {test.shape}")

In [ ]:
%matplotlib inline
src.visualise_data(train_5min[:,:,:,:,0])

#### BASIC CONVOLUTIONAL AUTOENCODER

##### USE MAXPOOLING AND UPSAMPLING

In [ ]:
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D
from keras.models import Model
from keras import backend as K

input_img = Input(shape=(64, 64, 1))

x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
encoded = BatchNormalization()(x)

x = Conv2D(128, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = Model(input_img, decoded)
autoencoder.summary()

In [ ]:
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
autoencoder.fit(overfit_train, overfit_truth,
                epochs=2000,
                batch_size=1,
                shuffle=True)
                #validation_data=(xval[:,0,:,:,:], xval[:,1,:,:,:]))

In [ ]:
hist = autoencoder.history
hist.history.keys()
plt.plot(hist.history['loss'])
# plt.plot(hist.history['val_loss'])
plt.grid()
plt.legend(['training', 'validation'], loc='upper left')
plt.xlabel("Epoch")
plt.ylabel("Loss")
#plt.yscale("log")
plt.savefig('training.png')

In [ ]:
overfit_prediction = autoencoder.predict(overfit_train, batch_size=1)
overfit_prediction.shape

In [ ]:
args = src.arg_getter(overfit_truth, overfit_prediction)
error_images, error_vals, error_means = src.error_distribution(overfit_truth, overfit_prediction, metric="relative_error")

In [ ]:
error_images.shape

In [ ]:
result_plotter(args, (overfit_train[:,:,:,0], overfit_truth[:,:,:,0], overfit_prediction[:,:,:,0], error_images[:,:,:,0]))

In [ ]:
from keras.callbacks import TensorBoard

autoencoder.compile(optimizer='adam', loss=custom_loss, metrics=[src.relative_error_tensor])
callback = [keras.callbacks.EarlyStopping(monitor='val_loss', patience=10),\
            keras.callbacks.ModelCheckpoint(filepath='cae_5min_50.h5', monitor='val_loss', save_best_only=True)]

autoencoder.fit(train_5min[:,0,:,:,:], train_5min[:,1,:,:,:],
                epochs=50,
                batch_size=100,
                shuffle=True,
                validation_data=(val_5min[:,0,:,:,:], val_5min[:,1,:,:,:]),
                callbacks=callback)

In [ ]:
# autoencoder.save("cae_50.h5")

In [ ]:
# autoencoder.load_weights("cae_50.h5")

In [ ]:
hist = autoencoder.history
hist.history.keys()

In [ ]:
plt.plot(hist.history['loss'],)
plt.plot(hist.history['val_loss'])
plt.grid()
plt.legend(['training', 'validation'], loc='upper left')
plt.xlabel("Epoch")
plt.ylabel("Loss")
#plt.yscale("log")
plt.savefig('training.png')

In [ ]:
predictions = autoencoder.predict(test[:,0,:,:,:], batch_size=100)
truth       = test[:,1,:,:,:]
truth.shape

In [ ]:
args = src.arg_getter(truth, predictions)
args[-1]

In [ ]:
error_images, error_vals, error_means = src.error_distribution(truth, predictions, metric="relative_error")

In [ ]:
test[:,0,:,:,0].shape, truth[:,:,:,0].shape, predictions[:,:,:,0].shape, error_images[:,:,:,0].shape

In [ ]:
def result_plotter(indices, datasets, task='prediction'):

    if task == 'prediction':
        title = ['Frame t', 'Frame t+1', 'Prediction t+1', 'Pixelwise difference']
    elif task == 'upsampling':
        title = ['Original', 'Downsampled', 'Upsampled', 'Pixelwise difference']
    else:
        sys.exit("Task must be 'prediction' or 'upsampling'.")
    for i in indices:
        fig, axes = plt.subplots(nrows=1, ncols=4, num=None, figsize=(16, 16), dpi=80, facecolor='w', edgecolor='k')
        for j, ax in enumerate(axes.flat):
            im = ax.imshow(datasets[j][int(i)], vmin=0,
                           vmax=max([np.max(dset[int(i)]) for dset in datasets[:2]]) if int(j) < 3 else None)
                           #, norm=colors.PowerNorm(gamma=0.5) if int(j) == 3 else None)
            ax.set_title(f"{title[j]}", fontsize=10)
            colorbar(im)
            ax.axis('off')
        plt.savefig(f"Sample_{i}.png")
    plt.show()
    
def colorbar(mappable):
    from mpl_toolkits.axes_grid1 import make_axes_locatable
    ax = mappable.axes
    fig = ax.figure
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    return fig.colorbar(mappable, cax=cax)


In [ ]:
result_plotter(args[:10], (test[:,0,:,:,0], truth[:,:,:,0], predictions[:,:,:,0], error_images[:,:,:,0]))

##### 5MIN RESOLUTION, ADD GRAD DIFF LOSS

In [ ]:
hist = autoencoder.history
hist.history.keys()

In [ ]:
plt.plot(hist.history['loss'],)
plt.plot(hist.history['val_loss'])
plt.grid()
plt.legend(['training', 'validation'], loc='upper left')
plt.xlabel("Epoch")
plt.ylabel("Loss")
#plt.yscale("log")
plt.savefig('training.png')

In [ ]:
predictions = autoencoder.predict(test_5min[:,0,:,:,:], batch_size=100)
truth       = test_5min[:,1,:,:,:]
truth.shape

In [ ]:
args = src.arg_getter(truth, predictions)
error_images, error_vals, error_means = src.error_distribution(truth, predictions, metric="difference")

In [ ]:
args[-1]

In [ ]:
result_plotter(args[:5], (test_5min[:,0,:,:,0], truth[:,:,:,0], predictions[:,:,:,0], error_images[:,:,:,0]))

##### SAME NETWORK WITH LSTM BUT NO LSTM LAYER.

In [ ]:
from keras.layers import Activation, BatchNormalization, Conv2DTranspose

input_img = Input(shape=(64, 64, 1))

x = Conv2D(8, (5, 5), strides=2, activation='relu', padding='same')(input_img)
x = BatchNormalization()(x)
x = Conv2D(16, (5, 5), strides=2, activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(32, (5, 5), strides=2, activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(64, (5, 5), strides=2, activation='relu', padding='same')(x)
x = BatchNormalization()(x)
encoded =  Conv2D(128, (5, 5), strides=4, activation='tanh', padding='same')(x)

x = Conv2DTranspose(64, (5, 5), strides=4, activation='relu', padding='same')(encoded)
x = Conv2DTranspose(32, (5, 5), strides=2, activation='relu', padding='same')(x)
x = Conv2DTranspose(16, (5, 5), strides=2, activation='relu', padding='same')(x)
x = Conv2DTranspose(8, (5, 5), strides=2, activation='relu', padding='same')(x)
decoded = Conv2DTranspose(1, (5, 5), strides=2, activation='relu', padding='same')(x)

ae_no_lstm = Model(input_img, decoded)
ae_no_lstm.summary()

In [ ]:
def gradient_diff(yTrue, yPred):
    alpha = 1
    if len(yTrue.shape) == 5 :
        true = K.pow(K.flatten(K.abs(K.abs(yTrue[:,:,1:,:,:] - yTrue[:,:,:-1,:,:]) -
                                     K.abs(yPred[:,:,1:,:,:] - yPred[:,:,:-1,:,:]))),alpha)
        pred = K.pow(K.flatten(K.abs(K.abs(yTrue[:,:,:,1:,:] - yTrue[:,:,:,:-1,:]) -
                                     K.abs(yPred[:,:,:,1:,:] - yPred[:,:,:,:-1,:]))),alpha)
    elif len(yTrue.shape) == 4 :
        true = K.pow(K.flatten(K.abs(K.abs(yTrue[:,1:,:,:] - yTrue[:,:-1,:,:]) -
                                     K.abs(yPred[:,1:,:,:] - yPred[:,:-1,:,:]))),alpha)
        pred = K.pow(K.flatten(K.abs(K.abs(yTrue[:,:,1:,:] - yTrue[:,:,:-1,:]) -
                                     K.abs(yPred[:,:,1:,:] - yPred[:,:,:-1,:]))),alpha)
    num = K.sum(true + pred)
    return num / tf.to_float((K.shape(true)[0] + K.shape(pred)[0]))

In [ ]:
#callback = [keras.callbacks.EarlyStopping(monitor='val_loss', patience=10),
#            keras.callbacks.ModelCheckpoint(filepath='ae_best.h5', monitor='val_loss', save_best_only=True)]

def custom_loss(y_true, y_pred):
    return keras.losses.mean_squared_error(y_true, y_pred) + gradient_diff(y_true, y_pred)

# autoencoder.load_weights("ae_best.h5")
ae_no_lstm.compile(optimizer='adam', loss='mean_squared_error', metrics=[src.relative_error_tensor])

In [ ]:
from keras.callbacks import TensorBoard

ae_no_lstm.fit(train[:,0,:,:,:], train[:,1,:,:,:],
                epochs=50,
                batch_size=100,
                shuffle=True,
                validation_data=(xval[:,0,:,:,:], xval[:,1,:,:,:]))

In [ ]:
hist = ae_no_lstm.history
plt.plot(hist.history['loss'],)
plt.plot(hist.history['val_loss'])
plt.grid()
plt.legend(['training', 'validation'], loc='upper left')
plt.xlabel("Epoch")
plt.ylabel("Loss")
#plt.yscale("log")
plt.savefig('training.png')

In [ ]:
predictions = ae_no_lstm.predict(test[:,0,:,:,:], batch_size=100)
truth       = test[:,1,:,:,:]
truth.shape

In [ ]:
args = src.arg_getter(truth, predictions)
args[-1]

In [ ]:
error_images, error_vals, error_means = src.error_distribution(truth, predictions, metric="relative_error")

In [ ]:
result_plotter(args[:10], (test[:,0,:,:,0], truth[:,:,:,0], predictions[:,:,:,0], error_images[:,:,:,0]))

#### VARIATIONAL AUTOENCODER

In [ ]:
from keras.layers import Lambda, Flatten

batch_size = 128
latent_dim = 32
epsilon_std = 0.0001

input_img = Input(batch_shape=(batch_size, 64, 64, 1))

x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

flat_x = Flatten()(encoded)
z_mean = Dense(latent_dim)(flat_x)
z_log_sigma = Dense(latent_dim)(flat_x)

def sampling(args):
    z_mean, z_log_sigma = args
    epsilon = K.random_normal(shape=(batch_size, latent_dim),
                              mean=0., stddev=epsilon_std)
    return z_mean + K.exp(z_log_sigma) * epsilon

# note that "output_shape" isn't necessary with the TensorFlow backend
# so you could write `Lambda(sampling)([z_mean, z_log_sigma])`
z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_sigma])
z = Dense(4*4*8)(z)
tf.reshape(z, [128, 4, 4, 8])

x = Conv2D(8, (3, 3), activation='relu', padding='same')(z)
x = UpSampling2D((2, 2))(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

vae = Model(input_img, decoded)
def vae_loss(x, x_decoded_mean):
    xent_loss = objectives.binary_crossentropy(x, x_decoded_mean)
    kl_loss = - 0.5 * K.mean(1 + z_log_sigma - K.square(z_mean) - K.exp(z_log_sigma), axis=-1)
    return xent_loss + kl_loss

vae.compile(optimizer='rmsprop', loss=vae_loss)
vae.summary()